In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from typing import Any, Dict, List, Tuple
from pathlib import Path
import yaml

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pandas_profiling import ProfileReport # profile report を作る用
from matplotlib_venn import venn2 # venn図を作成する用
from tqdm import tqdm
from contextlib import contextmanager
from time import time

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

from mykaggle.metric.mse import rmsle
from mykaggle.util.ml_logger import MLLogger
from mykaggle.lib.lgbm_util import compute_importances, save_importances
from mykaggle.lib.pandas_util import change_column_name
from mykaggle.util.routine import fix_seed

sns.set_style('darkgrid')

In [2]:
settings = yaml.safe_load('''
name: '018_eda'
competition: atmacup8
seed: 1019
training:
    num_folds: 5
    num_rounds: 1000
    early_stopping_rounds: 100
    verbose_eval: 20
lgbm_params:
    objective: binary
    learning_rate: 0.05
    max_depth: -1
    num_leaves: 31
    colsample_bytree: .7
    metric: "None"
''')

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
fix_seed(settings['seed'])

In [4]:
datadir = Path('../data/')
ckptdir = Path('../ckpt/') / settings['name']
if not ckptdir.exists():
    ckptdir.mkdir()

In [5]:
df_train = pd.read_csv(datadir / 'id_train.csv')
df_test = pd.read_csv(datadir / 'id_test.csv')
df_submission = pd.read_csv(datadir / 'atmaCup8_sample-submission.csv')
df_train.shape, df_test.shape

((8359, 17), (8360, 12))

In [6]:
df_train.head().T

,0,1,2,3,4
Name,LEGO Batman: The Videogame,LEGO Indiana Jones: The Original Adventures,LEGO Batman: The Videogame,Combat,LEGO Harry Potter: Years 5-7
Platform,Wii,Wii,PSP,2600,Wii
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Warner Bros. Interactive Entertainment,LucasArts,Warner Bros. Interactive Entertainment,Atari,Warner Bros. Interactive Entertainment
NA_Sales,180,151,56,117,69
EU_Sales,97,61,44,7,42
JP_Sales,0,0,0,0,0
Other_Sales,28,21,27,1,12
Global_Sales,306,234,128,125,124


In [7]:
df_test.head().T

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4
Name,Hitman 2: Silent Assassin,Legacy of Kain: Soul Reaver,Metal Gear Solid 2: Substance,Silent Hill: Homecoming,Silent Hill: Homecoming
Platform,XB,PS,XB,X360,PS3
Year_of_Release,NaN,NaN,NaN,NaN,NaN
Genre,Action,Action,Action,Action,Action
Publisher,Eidos Interactive,Eidos Interactive,Konami Digital Entertainment,Konami Digital Entertainment,Konami Digital Entertainment
Critic_Score,84,91,87,70,71
Critic_Count,23,17,28,54,41
User_Score,8,9,8.5,6.9,6.9
User_Count,19,132,39,180,143
Developer,Io Interactive,Crystal Dynamics,KCEJ,Double Helix Games,Double Helix Games


In [8]:
df_submission.head()

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Global_Sales
0,63.371815
1,63.371815
2,63.371815
3,63.371815
4,63.371815


# なぜか index が効くのでなぜか考える

In [9]:
for i in range(df_train.shape[0] // 40 + 1):
    display(df_train.iloc[i * 40:(i + 1) * 40]['Year_of_Release'])

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
10   NaN
11   NaN
12   NaN
13   NaN
14   NaN
15   NaN
16   NaN
17   NaN
18   NaN
19   NaN
20   NaN
21   NaN
22   NaN
23   NaN
24   NaN
25   NaN
26   NaN
27   NaN
28   NaN
29   NaN
30   NaN
31   NaN
32   NaN
33   NaN
34   NaN
35   NaN
36   NaN
37   NaN
38   NaN
39   NaN
Name: Year_of_Release, dtype: float64

40   NaN
41   NaN
42   NaN
43   NaN
44   NaN
45   NaN
46   NaN
47   NaN
48   NaN
49   NaN
50   NaN
51   NaN
52   NaN
53   NaN
54   NaN
55   NaN
56   NaN
57   NaN
58   NaN
59   NaN
60   NaN
61   NaN
62   NaN
63   NaN
64   NaN
65   NaN
66   NaN
67   NaN
68   NaN
69   NaN
70   NaN
71   NaN
72   NaN
73   NaN
74   NaN
75   NaN
76   NaN
77   NaN
78   NaN
79   NaN
Name: Year_of_Release, dtype: float64

80        NaN
81        NaN
82        NaN
83        NaN
84     1980.0
85     1980.0
86     1980.0
87     1980.0
88     1981.0
89     1981.0
90     1981.0
91     1981.0
92     1981.0
93     1981.0
94     1981.0
95     1981.0
96     1981.0
97     1981.0
98     1981.0
99     1981.0
100    1981.0
101    1981.0
102    1981.0
103    1981.0
104    1981.0
105    1981.0
106    1981.0
107    1981.0
108    1981.0
109    1981.0
110    1981.0
111    1981.0
112    1981.0
113    1981.0
114    1981.0
115    1981.0
116    1981.0
117    1981.0
118    1981.0
119    1981.0
Name: Year_of_Release, dtype: float64

120    1981.0
121    1981.0
122    1982.0
123    1982.0
124    1982.0
125    1982.0
126    1982.0
127    1982.0
128    1982.0
129    1982.0
130    1982.0
131    1982.0
132    1982.0
133    1982.0
134    1982.0
135    1982.0
136    1982.0
137    1982.0
138    1982.0
139    1982.0
140    1982.0
141    1982.0
142    1982.0
143    1982.0
144    1982.0
145    1983.0
146    1983.0
147    1983.0
148    1983.0
149    1983.0
150    1983.0
151    1983.0
152    1983.0
153    1983.0
154    1983.0
155    1983.0
156    1983.0
157    1983.0
158    1983.0
159    1984.0
Name: Year_of_Release, dtype: float64

...

8160    2016.0
8161    2016.0
8162    2016.0
8163    2016.0
8164    2016.0
8165    2016.0
8166    2016.0
8167    2016.0
8168    2016.0
8169    2016.0
8170    2016.0
8171    2016.0
8172    2016.0
8173    2016.0
8174    2016.0
8175    2016.0
8176    2016.0
8177    2016.0
8178    2016.0
8179    2016.0
8180    2016.0
8181    2016.0
8182    2016.0
8183    2016.0
8184    2016.0
8185    2016.0
8186    2016.0
8187    2016.0
8188    2016.0
8189    2016.0
8190    2016.0
8191    2016.0
8192    2016.0
8193    2016.0
8194    2016.0
8195    2016.0
8196    2016.0
8197    2016.0
8198    2016.0
8199    2016.0
Name: Year_of_Release, dtype: float64

8200    2016.0
8201    2016.0
8202    2016.0
8203    2016.0
8204    2016.0
8205    2016.0
8206    2016.0
8207    2016.0
8208    2016.0
8209    2016.0
8210    2016.0
8211    2016.0
8212    2016.0
8213    2016.0
8214    2016.0
8215    2016.0
8216    2016.0
8217    2016.0
8218    2016.0
8219    2016.0
8220    2016.0
8221    2016.0
8222    2016.0
8223    2016.0
8224    2016.0
8225    2016.0
8226    2016.0
8227    2016.0
8228    2016.0
8229    2016.0
8230    2016.0
8231    2016.0
8232    2016.0
8233    2016.0
8234    2016.0
8235    2016.0
8236    2016.0
8237    2016.0
8238    2016.0
8239    2016.0
Name: Year_of_Release, dtype: float64

8240    2016.0
8241    2016.0
8242    2016.0
8243    2016.0
8244    2016.0
8245    2016.0
8246    2016.0
8247    2016.0
8248    2016.0
8249    2016.0
8250    2016.0
8251    2016.0
8252    2016.0
8253    2016.0
8254    2016.0
8255    2016.0
8256    2016.0
8257    2016.0
8258    2016.0
8259    2016.0
8260    2016.0
8261    2016.0
8262    2016.0
8263    2016.0
8264    2016.0
8265    2016.0
8266    2016.0
8267    2016.0
8268    2016.0
8269    2016.0
8270    2016.0
8271    2016.0
8272    2016.0
8273    2016.0
8274    2016.0
8275    2016.0
8276    2016.0
8277    2016.0
8278    2016.0
8279    2016.0
Name: Year_of_Release, dtype: float64

8280    2016.0
8281    2016.0
8282    2016.0
8283    2016.0
8284    2016.0
8285    2016.0
8286    2016.0
8287    2016.0
8288    2016.0
8289    2016.0
8290    2016.0
8291    2016.0
8292    2016.0
8293    2016.0
8294    2016.0
8295    2016.0
8296    2016.0
8297    2016.0
8298    2016.0
8299    2016.0
8300    2016.0
8301    2016.0
8302    2016.0
8303    2016.0
8304    2016.0
8305    2016.0
8306    2016.0
8307    2016.0
8308    2016.0
8309    2016.0
8310    2016.0
8311    2016.0
8312    2016.0
8313    2016.0
8314    2016.0
8315    2016.0
8316    2016.0
8317    2016.0
8318    2016.0
8319    2016.0
Name: Year_of_Release, dtype: float64

8320    2016.0
8321    2016.0
8322    2016.0
8323    2016.0
8324    2016.0
8325    2016.0
8326    2016.0
8327    2016.0
8328    2016.0
8329    2016.0
8330    2016.0
8331    2016.0
8332    2016.0
8333    2016.0
8334    2016.0
8335    2016.0
8336    2016.0
8337    2016.0
8338    2016.0
8339    2016.0
8340    2016.0
8341    2016.0
8342    2016.0
8343    2016.0
8344    2016.0
8345    2016.0
8346    2016.0
8347    2016.0
8348    2016.0
8349    2016.0
8350    2016.0
8351    2016.0
8352    2016.0
8353    2016.0
8354    2016.0
8355    2016.0
8356    2017.0
8357    2017.0
8358    2017.0
Name: Year_of_Release, dtype: float64

In [22]:
year_count = df_train.groupby('Year_of_Release')['id'].count().reset_index()
year_count = change_column_name(year_count, 'id', 'year_count')

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
year_rank = df_train.groupby('Year_of_Release')['id'].rank()
df_train['year_rank'] = year_rank

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
df_train = pd.merge(df_train, year_count.loc[:, ['Year_of_Release', 'year_count']], how='left', on='Year_of_Release')
df_train.loc[df_train.loc[:, 'year_count'].isna(), 'year_rank'] = np.nan
df_train['year_rank_rate'] = df_train.loc[:, 'year_rank'] / df_train.loc[:, 'year_count']
df_train

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,year_rank,year_count,year_rank_rate
0,LEGO Batman: The Videogame,Wii,NaN,Action,Warner Bros. Interactive Entertainment,180,97,0,28,306,74.0,17.0,7.9,22.0,Traveller's Tales,E10+,0,NaN,NaN,NaN
1,LEGO Indiana Jones: The Original Adventures,Wii,NaN,Action,LucasArts,151,61,0,21,234,78.0,22.0,6.6,28.0,Traveller's Tales,E10+,1,NaN,NaN,NaN
2,LEGO Batman: The Videogame,PSP,NaN,Action,Warner Bros. Interactive Entertainment,56,44,0,27,128,73.0,5.0,7.4,10.0,Traveller's Tales,E10+,2,NaN,NaN,NaN
3,Combat,2600,NaN,Action,Atari,117,7,0,1,125,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
4,LEGO Harry Potter: Years 5-7,Wii,NaN,Action,Warner Bros. Interactive Entertainment,69,42,0,12,124,76.0,8.0,7.8,13.0,Traveller's Tales,E10+,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8354,Stellaris,PC,2016.0,Strategy,Paradox Interactive,0,4,0,0,4,78.0,57.0,8,569.0,Paradox Development Studio,NaN,8354,259.0,260.0,0.996154
8355,Total War Attila: Tyrants & Kings,PC,2016.0,Strategy,Koch Media,0,1,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,8355,260.0,260.0,1.000000
8356,Brothers Conflict: Precious Baby,PSV,2017.0,Action,Idea Factory,0,0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,8356,1.0,3.0,0.333333
8357,Phantasy Star Online 2 Episode 4: Deluxe Package,PS4,2017.0,Role-Playing,Sega,0,0,4,0,4,NaN,NaN,NaN,NaN,NaN,NaN,8357,2.0,3.0,0.666667


# train/test 

In [10]:
train = df_train.copy()
test = df_test.copy()

train.loc[:, 'is_test'] = False
test.loc[:, 'is_test'] = True
whole = pd.concat([train, test])
whole.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,id,is_test
0,LEGO Batman: The Videogame,Wii,NaN,Action,Warner Bros. Interactive Entertainment,180.0,97.0,0.0,28.0,306.0,74.0,17.0,7.9,22.0,Traveller's Tales,E10+,0,False
1,LEGO Indiana Jones: The Original Adventures,Wii,NaN,Action,LucasArts,151.0,61.0,0.0,21.0,234.0,78.0,22.0,6.6,28.0,Traveller's Tales,E10+,1,False
2,LEGO Batman: The Videogame,PSP,NaN,Action,Warner Bros. Interactive Entertainment,56.0,44.0,0.0,27.0,128.0,73.0,5.0,7.4,10.0,Traveller's Tales,E10+,2,False
3,Combat,2600,NaN,Action,Atari,117.0,7.0,0.0,1.0,125.0,NaN,NaN,NaN,NaN,NaN,NaN,3,False
4,LEGO Harry Potter: Years 5-7,Wii,NaN,Action,Warner Bros. Interactive Entertainment,69.0,42.0,0.0,12.0,124.0,76.0,8.0,7.8,13.0,Traveller's Tales,E10+,4,False


In [11]:
df_base_train = df_train.loc[:, ['id']]
df_base_test = df_test.loc[:, ['id']]

train_others = {
    'main': df_train.copy(),
    'another': df_test.copy()
}
test_others = {
    'main': df_test.copy(),
    'another': df_train.copy(),
}

/home/jupyter/workspace/atmacup8/.venv/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## categry を見る

In [ ]:
def countplot(df: pd.DataFrame, column: str) -> None:
    print(f'{column}: {df.loc[:, column].unique()}')
    plt.figure(figsize=(20, 8))
    sns.countplot(data=df, x=column, hue='is_test')
    plt.title(column)
    plt.tight_layout()

In [ ]:
countplot(whole, 'Year_of_Release')

In [ ]:
np.sum(whole['Year_of_Release'].isna())

# Name をみる

In [ ]:
whole.loc[:, ['Name']].value_counts()

In [ ]:
whole[whole['Name'] == 'LEGO Marvel Super Heroes']

In [ ]:
le = LabelEncoder()
le.fit(whole['Platform'])
whole['le_Platform'] = le.transform(whole['Platform'])
platform_list = whole.groupby('Name')['le_Platform'].apply(list).reset_index()
platform_list = change_column_name(platform_list, 'le_Platform', 'other_platforms')
platform_list

In [ ]:
def get_other_platforms(x):
    if np.isnan(x['other_platforms']).any():
        return []
    return [p for p in x['other_platforms'] if p != x['le_Platform']]

In [ ]:
merge = pd.merge(whole, platform_list, how='left', on='Name')
merge['other_platforms'] = merge[['other_platforms', 'le_Platform']].apply(get_other_platforms, axis=1)
merge.head()

In [ ]:
num_platforms = len(merge['le_Platform'].unique())
onehot = np.zeros((merge.shape[0], num_platforms), dtype=np.int32)
for i, other_platforms in enumerate(merge['other_platforms']):
    for p in other_platforms:
        onehot[i, p] = 1

In [ ]:
df_onehot = pd.DataFrame(onehot, columns=[f'other_platform_{i+1}'for i in range(onehot.shape[1])])
merge = pd.concat([merge, df_onehot], axis=1)
merge.head()

In [ ]:
from mykaggle.feature.other_platforms import OtherPlatforms
other_platforms = OtherPlatforms(train=True)

In [ ]:
other_platforms(df_train.copy(), train_others)

Name がかぶってるやつは複数 Platform で出てるやつ?

1個じゃないやつは割とある (全体の半分)

In [ ]:
print('whole, eq 1, lt 1')
print(np.sum(whole.loc[:, ['Name']].value_counts() == 1), ',', np.sum(whole.loc[:, ['Name']].value_counts() != 1))
print('train test lt 1')
print(np.sum(train.loc[:, ['Name']].value_counts() != 1), np.sum(test.loc[:, ['Name']].value_counts() != 1))

In [ ]:
multi_platform = (train.loc[:, ['Name']].value_counts()).reset_index()
multi_platform = change_column_name(multi_platform, 0, 'multi_pf_count')
multi_platform['is_multi_platform'] = multi_platform.loc[:, 'multi_pf_count'] >= 1
multi_platform.head()

In [ ]:
train.loc[train.loc[:, ['Name']].value_counts() != 1, 'Name']

Name 被ってて train/test に別れてる (=Publisher が別)があるけどなんだ?

In [ ]:
name_duplicated = whole.groupby('Name')['is_test'].mean().reset_index()
name_duplicated.head()

In [ ]:
name_duplicated['is_not_1'] = name_duplicated['is_test'] != 1.0
name_duplicated['is_not_0'] = name_duplicated['is_test'] != 0.0
name_duplicated['is_not_01'] = name_duplicated['is_not_1'] * name_duplicated['is_not_0']
name_duplicated.shape

In [ ]:
name_duplicated[name_duplicated['is_not_01']]

In [ ]:
whole.loc[whole['Name'] == '428: Fuusa Sareta Shibuya de']

1つのソフトでPublisher はまたがることがある

# Score, Count をみる

In [ ]:
train = df_train.copy()
test = df_test.copy()

train.loc[:, 'is_test'] = False
test.loc[:, 'is_test'] = True
whole = pd.concat([train, test])

In [ ]:
train.loc[:, 'User_Count']

In [ ]:
train.loc[:, 'User_Score'] / 10.0

In [ ]:
# nan の割合
print('User_Score train:', train[train['User_Score'].isna()].shape[0], train[train['User_Score'].isna()].shape[0] / train.shape[0])
print('User_Score test:', test[test['User_Score'].isna()].shape[0], test[test['User_Score'].isna()].shape[0] / test.shape[0])
print('Critic_Score train:', train[train['Critic_Score'].isna()].shape[0], train[train['Critic_Score'].isna()].shape[0] / train.shape[0])
print('Critic_Score test:', test[test['Critic_Score'].isna()].shape[0], test[test['Critic_Score'].isna()].shape[0] / test.shape[0])

In [ ]:
# 数字でも NaN でもないやつある? -> ある。tbd が 1132 個
train[~train['User_Score'].isna() & train['User_Score'].str.isalpha()]['User_Score'].value_counts()

In [ ]:
# train[~train['Critic_Score'].isna() & train['Critic_Score'].str.isalpha()]['Critic_Score'].value_counts()
# Critic_Score にはない

In [ ]:
# tbd をみる。User_Count は nan になる
train[~train['User_Score'].isna() & train['User_Score'].str.isalpha()].head()

In [ ]:
# tbd と nan 何が違う?
train[train['User_Score'].isna()]

In [ ]:
train.loc[train['User_Score'] == 'tbd', 'User_Score'] = np.nan

In [ ]:
train['User_Score'].astype(np.float32)

In [ ]:
train['User_Score'].astype(np.float32) / 10.0

In [ ]:
train['User_Score'] = train.loc[:, 'User_Score'].fillna(-1)
train['Critic_Score'] = train.loc[:, 'Critic_Score'].fillna(-1)

In [ ]:
print(train['User_Score'].astype(np.float32).describe())
sns.distplot(train['User_Score'])

In [ ]:
print(train['Critic_Score'].describe())
sns.distplot(train['Critic_Score'])

In [ ]:
print(train['User_Count'].describe())
sns.distplot(train['User_Count'])

In [ ]:
print(train['Critic_Count'].describe())
sns.distplot(train['Critic_Count'])

In [ ]:
# Sales

In [ ]:
df_train.loc[:, 'Global_Sales'].describe()